# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/guoyalong/Desktop/homework/python-api-challenge/WeatherPy/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lompoc,34.6391,-120.4579,18.85,55,0,2.68,US,1699550126
1,1,alice springs,-23.7000,133.8833,18.75,48,76,3.09,AU,1699549991
2,2,isafjordur,66.0755,-23.1240,4.02,65,75,8.23,IS,1699550108
3,3,irymple,-34.2500,142.1667,20.59,36,1,3.11,AU,1699550240
4,4,mary,37.0000,62.5000,14.04,33,89,6.39,TM,1699550240


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points("Lng", 
                                 "Lat", 
                                  geo = True,
                                  color = "City",
                                  alpha = 0.6,
                                  size = "Humidity",
                                  tiles = "OSM")

# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_df.dropna()

# Display sample data
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,guerrero negro,27.9769,-114.0611,21.15,56,0,0.83,MX,1699549859
33,33,saint-pierre,-21.3393,55.4781,24.82,69,0,2.57,RE,1699550073
86,86,tamanrasset,22.7850,5.5228,24.95,10,0,2.06,DZ,1699550291
90,90,suwayr,30.1171,40.3893,24.52,26,0,2.70,SA,1699550294
106,106,mulayjah,27.2710,48.4242,26.23,45,0,2.85,SA,1699550304
107,107,holualoa,19.6228,-155.9522,21.54,81,0,2.68,US,1699550013
148,148,fort bragg,35.1390,-79.0060,25.46,56,0,3.60,US,1699550333
256,256,cannonvale,-20.2833,148.7000,22.66,74,1,4.47,AU,1699550278
266,266,lagonoy,13.7342,123.5207,24.87,94,2,1.02,PH,1699550408
361,361,kailua-kona,19.6406,-155.9956,23.61,79,0,2.68,US,1699550242


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,guerrero negro,MX,27.9769,-114.0611,56,
33,saint-pierre,RE,-21.3393,55.4781,69,
86,tamanrasset,DZ,22.7850,5.5228,10,
90,suwayr,SA,30.1171,40.3893,26,
106,mulayjah,SA,27.2710,48.4242,45,
107,holualoa,US,19.6228,-155.9522,81,
148,fort bragg,US,35.1390,-79.0060,56,
256,cannonvale,AU,-20.2833,148.7000,74,
266,lagonoy,PH,13.7342,123.5207,94,
361,kailua-kona,US,19.6406,-155.9956,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
guerrero negro - nearest hotel: Plaza sal paraiso
saint-pierre - nearest hotel: Tropic Hotel
tamanrasset - nearest hotel: فندق الأمان
suwayr - nearest hotel: No hotel found
mulayjah - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
fort bragg - nearest hotel: Airborne Inn Lodging
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
lagonoy - nearest hotel: Port Alfredo Beach and Hotel
kailua-kona - nearest hotel: Kona Seaside Hotel
abadan - nearest hotel: خانه معلم
sirhind - nearest hotel: No hotel found
shush - nearest hotel: هتل آپادانا
viedma - nearest hotel: Níjar
ujjain - nearest hotel: Meghdoot luxury hotel and resort
rancho palos verdes - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
10,guerrero negro,MX,27.9769,-114.0611,56,Plaza sal paraiso
33,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
86,tamanrasset,DZ,22.7850,5.5228,10,فندق الأمان
90,suwayr,SA,30.1171,40.3893,26,No hotel found
106,mulayjah,SA,27.2710,48.4242,45,No hotel found
107,holualoa,US,19.6228,-155.9522,81,Kona Hotel
148,fort bragg,US,35.1390,-79.0060,56,Airborne Inn Lodging
256,cannonvale,AU,-20.2833,148.7000,74,Whitsundays Rainforest Retreat
266,lagonoy,PH,13.7342,123.5207,94,Port Alfredo Beach and Hotel
361,kailua-kona,US,19.6406,-155.9956,79,Kona Seaside Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 750, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)